In [1]:
!pip install sbol2build

In [1]:
import sbol2build as s2b

In [ ]:
bsai = s2b.rebase_restriction_enzyme("BsaI")

In [3]:
bsai

In [ ]:
pro = s2b.dna_componentdefinition_with_sequence('promoter', 'acgt')

In [5]:
pro

(<sbol2.componentdefinition.ComponentDefinition at 0x7f872f7a4220>,
 <sbol2.sequence.Sequence at 0x7f872f7a4160>)

In [7]:
pro[1].elements

'acgt'